## Task 1 : Setting Up Enviornment

In [ ]:
!pip install torch torchvision
!git clone https://github.com/parth1620/Project-NST.git

## Task 2 : Loading VGG Pretrained Model

In [53]:
content_route = "path/to/content_image"
style_route = "path/to/style_image"

In [ ]:
import torch
from torchvision import models
vgg=models.vgg19(pretrained=True)
print(vgg)

In [55]:
vgg=vgg.features

In [56]:
for parameters in vgg.parameters():
  parameters.requires_grad_(False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
vgg.to(device)

## Task 3 : Preprocess image

Torchvision models page : https://pytorch.org/docs/stable/torchvision/models.html

In [59]:
from PIL import Image
from torchvision import transforms as T

In [60]:
def preprocess(img_path,max_size=500):
  image = Image.open(img_path).convert("RGB")
  if max(image.size)>max_size:
    size=max_size
  else:
    size=image.size
  img_transforms = T.Compose([
      T.Resize(size),
      T.ToTensor(),
      T.Normalize(mean=[0.485,0.456,0.406],
                  std=[0.229,0.224,0.225])
      ])

  image = img_transforms(image)
  image = image.unsqueeze(0)
  return image



In [ ]:
content_p=preprocess(content_route)
content_s=preprocess(style_route)

content_p=content_p.to(device)
style_p=content_s.to(device)
print(content_p.shape)
print(content_s.shape)

## Task 4 : Deprocess image

In [62]:
import numpy as np
import matplotlib.pyplot as plt

def deprocess(tensor):
  image= tensor.to('cpu').clone()
  image = image.numpy()
  image=image.squeeze(0)
  image=image.transpose(1,2,0)
  image = image*np.array([0.229,0.224,0.225]) + np.array([0.485,0.456,0.406])
  image= image.clip(0,1)
  return image


In [ ]:
content_d=deprocess(content_p)
style_d=deprocess(content_s)

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(20,10))
ax1.imshow(content_d)
ax2.imshow(style_d)

## Task 5 : Get content,style features and create gram matrix

In [64]:
def get_features(image,model):
  layers={
      '0': 'conv1_1',
      '5': 'conv2_1',
      '10': 'conv3_1',
      '19': 'conv4_1',
      '21': 'conv4_2',
      '28': 'conv5_1'
  }

  x=image
  features={}
  for name,layer in model._modules.items():
    x=layer(x)
    if name in layers:
      features[layers[name]]=x
  return features

In [65]:
content_f=get_features(content_p,vgg)
style_f=get_features(style_p,vgg)

In [66]:
def gram_matrix(tensor):
  b,c,h,w=tensor.size()
  tensor = tensor.view(c,h*w)
  gram = torch.mm(tensor,tensor.t())
  return gram

In [67]:
style_grams={layer: gram_matrix(style_f[layer]) for layer in style_f}

## Task 6 : Creating Style and Content loss function

In [68]:
def content_loss(content,target):
  loss=torch.mean((content-target)**2)
  return loss

In [69]:
style_weights={
    'conv1_1': 0.75,
    'conv2_1': 0.5,
    'conv3_1': 0.1,
    'conv4_1': 0.1,
    'conv5_1': 0.05,

}

In [70]:
def style_loss(style_weights,target_features,style_grams):
  loss=0
  for layer in style_weights:
    target_f = target_features[layer]
    target_gram = gram_matrix(target_f)
    style_gram=style_grams[layer]
    b,c,h,w=target_f.shape
    layer_loss=style_weights[layer]*torch.mean((style_gram-target_gram)**2)
    loss+=layer_loss
  return loss

In [ ]:
target=content_p.clone().requires_grad_(True).to(device)
target_f=get_features(target,vgg)
print(content_loss(target_f['conv4_2'],content_f['conv4_2']))
print(style_loss(style_weights,target_f,style_grams))

## Task 7 : Training loop

In [72]:
from torch import optim
optimizer=optim.Adam([target],lr=0.002)
alpha=1
beta=4
tv_weight=1e-6

epochs=3501
show_every=500

In [73]:
def total_loss(c_loss, s_loss, tv_loss, alpha, beta, tv_weight):
    loss = alpha * c_loss + beta * s_loss + tv_weight * tv_loss
    return loss


def total_variation_loss(image):
    tv_loss = torch.sum(torch.abs(image[:, :, :-1] - image[:, :, 1:])) + torch.sum(torch.abs(image[:, :-1, :] - image[:, 1:, :]))
    return tv_loss

In [ ]:
result = []

for i in range(epochs):
    target_f = get_features(target, vgg)
    c_loss = content_loss(target_f['conv4_2'], content_f['conv4_2'])
    s_loss = style_loss(style_weights, target_f, style_grams)
    tv_loss = total_variation_loss(target)
    loss = total_loss(c_loss, s_loss, tv_loss, alpha, beta, tv_weight)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % show_every == 0:
        print(f"Loss after epoch {i}: {loss.item()}")
        result.append(deprocess(target.detach()))


In [ ]:
plt.figure(figsize=(10, 8))
for i in range(len(result)):
    plt.subplot(4, 2, i + 1)
    plt.imshow(result[i])
plt.show()

print("Final results are:")


## Task 8 : Final Result

In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(1,2,1)
plt.imshow(content_d)
plt.title('Initial Image')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(result[-1])
plt.title('Final Image')
plt.axis('off')

plt.show()